In [99]:
import numpy as np

In [100]:

pose = [105, 340, 40, 1.87]
#pose = [242.82, 16.41, 48.48, 2.97]
#pose = [157.67, 136.43, 45.02, 3.04]
pose = [175, 199, 290, 3.14]

In [101]:
dh_params = np.array([[0,-1.5708,103.91,1.5708],
            [205.73,0.0,0.0,-1.3342],
            [200,0.0,0, 1.2888],
            [0.0,1.5708,0.0,1.5708],
            [0.0,0.0,131,0.0]])

In [102]:


x,y,z,psi = pose
psi = psi-np.deg2rad(90)

l1 = dh_params[0][2]
l2 = dh_params[1][0]
l3 = dh_params[2][0]
l4 = dh_params[4][2]

theta1 = np.arctan2(-x,y)
theta5 = 0


D = np.sqrt(x**2+y**2)
D_hat = D - l4*np.cos(psi)
z_hat = z + l4*np.sin(psi)
z_hat = z_hat 


r = np.sqrt(D_hat**2+z_hat**2)

value = ((l2**2 + l3**2 - r**2)/(2*l2*l3)%2 )/2


elbow = np.arccos(value)



theta3 = np.pi/2 + np.arctan2(50, 200) - elbow


value = ((r**2 + l2**2 - l3**2)/(2*r*l2) % 2 )/2

theta2= np.pi/2 - np.arctan2(50, 200) - np.arctan2(z_hat,D_hat) - np.arccos(value)

theta4 = psi - (theta2 + theta3)

#theta3 -= np.arctan2(200, 50)
#theta2 -= np.arctan2(50, 200)
print("theta1")
print(np.rad2deg(theta1))
print("theta2")
print(np.rad2deg(theta2))
print("theta3")
print(np.rad2deg(theta3))
print("theta4")
print(np.rad2deg(theta4))
pos = [theta1, theta2, theta3, theta4, theta5]
#pos = [theta1, 21.18, 10.9, 62.23, theta5]
value

theta1
-41.32829990162929
theta2
-34.274635009124594
theta3
99.81019046411235
theta4
24.37319221609074


0.6100517008517157

In [103]:
def dh_matrix_list_gen(theta, d, a, alpha):
    if not (len(theta) == len(d) == len(a) == len(alpha)):
        raise ValueError("non-equal dh params.")
    
    n = len(theta)
    dh_matrices = []

    for i in range(n):
        matrix = np.array([
            [np.cos(theta[i]), -np.sin(theta[i]) * np.cos(alpha[i]),  np.sin(theta[i]) * np.sin(alpha[i]), a[i] * np.cos(theta[i])],
            [np.sin(theta[i]),  np.cos(theta[i]) * np.cos(alpha[i]), -np.cos(theta[i]) * np.sin(alpha[i]), a[i] * np.sin(theta[i])],
            [0,                 np.sin(alpha[i]),                   np.cos(alpha[i]),                   d[i]],
            [0,                 0,                                  0,                                  1]
        ])
        dh_matrices.append(matrix)

    return np.array(dh_matrices)

In [104]:
def comp_forward_kin(dh_matrices):
    if not isinstance(dh_matrices, np.ndarray) or len(dh_matrices.shape) != 3 or dh_matrices.shape[1:] != (4, 4):
        raise ValueError("Input must be a np array with shape (n, 4, 4).")
    
    cumulative_matrix = np.eye(4)
    
    for i, matrix in enumerate(dh_matrices):
        cumulative_matrix = np.dot(cumulative_matrix, matrix)
        
        position = cumulative_matrix[:3, 3]
        #print(f"Step {i+1}: x = {position[0]:.4f}, y = {position[1]:.4f}, z = {position[2]:.4f}")
    return cumulative_matrix

In [105]:
def FK_dh(dh_params, joint_angles, link):
    """!
    @brief      Get the 4x4 transformation matrix from link to world

                TODO: implement this function

                Calculate forward kinematics for rexarm using DH convention

                return a transformation matrix representing the pose of the desired link

                note: phi is the euler angle about the y-axis in the base frame

    @param      dh_params     The dh parameters as a 2D list each row represents a link and has the format [a, alpha, d,
                              theta]
    @param      joint_angles  The joint angles of the links
    @param      link          The link to transform from

    @return     a transformation matrix representing the pose of the desired link
    """

    theta = dh_params[:,3]
    a = dh_params[:,0]
    alpha = dh_params[:,1]
    d = dh_params[:,2]

    theta = theta + joint_angles

    dh_matrix_list = dh_matrix_list_gen(theta, d, a, alpha)

    return comp_forward_kin(dh_matrix_list)

In [106]:
def get_pose_from_T(T):
    """!
    @brief      Gets the pose from T.

                TODO: implement this function return the 6DOF pose vector from a 4x4 transformation matrix T

    @param      T     transformation matrix

    @return     The pose vector from T.
    """

    position = T[:3, 3]
    phi = np.arctan2(T[1,2],-T[0,2])
    theta = np.arccos(T[2,2])
    psi = np.arctan2(T[2,1], T[2,0])

    return [position[0], position[1], position[2], phi, theta, psi]

In [107]:
ee_T = FK_dh(dh_params, pos, 5)
ee_pose = get_pose_from_T(ee_T)
print("ee_pose")
print(ee_pose)

pose = [105, 340, 40, 1.87]
pose = [242.82, 16.41, 48.48, 2.97]
#pose = [157.67, 136.43, 45.02, 3.04]

ee_pose
[16.09556257558835, 18.30171275578283, -12.617317479743946, 2.292263875612213, 3.094603672630925, -3.141436339102452]
